## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-14-16-50-00 +0000,wsj,"Goldman Sachs Profit Surges, Fueled by Dealmak...",https://www.wsj.com/business/earnings/goldman-...
1,2025-10-14-16-48-00 +0000,wsj,Torment Goes On for Families of Hostages Fight...,https://www.wsj.com/world/middle-east/torment-...
2,2025-10-14-16-47-59 +0000,wapo,Community colleges are losing millions in fund...,https://www.washingtonpost.com/nation/2025/10/...
3,2025-10-14-16-47-58 +0000,nyt,Newsom Vetoes California’s Ban on ‘Forever Che...,https://www.nytimes.com/2025/10/14/climate/pfa...
4,2025-10-14-16-47-50 +0000,bbc,Miliband defends clean power goal after energy...,https://www.bbc.com/news/articles/ckgzn1vx701o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2283/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,49
61,gaza,23
400,china,17
70,new,17
108,deal,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
304,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet about Trump’s role in histo...,https://nypost.com/2025/10/13/us-news/top-dems...,125
81,2025-10-14-14-24-22 +0000,nypost,Kamala Harris refuses to name Trump as she lau...,https://nypost.com/2025/10/14/us-news/kamala-h...,124
329,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...,124
298,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...,123
341,2025-10-13-17-58-10 +0000,nypost,Trump boasts Gaza peace process could be presi...,https://nypost.com/2025/10/13/us-news/trump-sa...,123


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
304,125,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet about Trump’s role in histo...,https://nypost.com/2025/10/13/us-news/top-dems...
74,59,2025-10-14-14-41-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
329,59,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...
275,56,2025-10-13-21-59-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
232,56,2025-10-14-01-59-18 +0000,cbc,"New York Times, AP, Reuters and Newsmax among ...",https://www.cbc.ca/news/world/new-york-times-a...
111,54,2025-10-14-13-00-00 +0000,wsj,"Tariffs, inflation and other threats such as e...",https://www.wsj.com/economy/trade-tensions-wei...
83,42,2025-10-14-14-20-00 +0000,wsj,China’s Rare-Earth Export Controls Are ‘Critic...,https://www.wsj.com/economy/trade/chinas-rare-...
238,39,2025-10-14-01-00-00 +0000,wsj,"The White House move to lay off roughly 4,000 ...",https://www.wsj.com/politics/policy/trump-gove...
32,37,2025-10-14-16-12-37 +0000,bbc,US Supreme Court rejects Alex Jones appeal in ...,https://www.bbc.com/news/articles/c5ypjjxd40qo...
173,35,2025-10-14-09-39-01 +0000,nypost,‘Heartbroken’ Mitt Romney speaks out after sis...,https://nypost.com/2025/10/14/us-news/mitt-rom...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
